In [1]:
import pyspark
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import split, create_map, lit

from itertools import chain

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("DE-project") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/01 00:21:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read \
    .option("header", "true") \
    .csv('/Users/alvin/Documents/projects/project_de-zoomcamp/01-ingestion/data/us_employment_2021.csv')

In [4]:
df.show()

+---------------+----------+-----------+--------------------+----+-----------+--------+----------+-----------------+-----+
|      LAUS_code|state_fips|county_fips|        county_state|year|labor_force|employed|unemployed|unemployment_rate| fips|
+---------------+----------+-----------+--------------------+----+-----------+--------+----------+-----------------+-----+
|CN0100100000000|        01|        001|  Autauga County, AL|2021|      26341|   25599|       742|              2.8|01001|
|CN0100300000000|        01|        003|  Baldwin County, AL|2021|      99427|   96481|      2946|              3.0|01003|
|CN0100500000000|        01|        005|  Barbour County, AL|2021|       8197|    7728|       469|              5.7|01005|
|CN0100700000000|        01|        007|     Bibb County, AL|2021|       8560|    8262|       298|              3.5|01007|
|CN0100900000000|        01|        009|   Blount County, AL|2021|      25127|   24529|       598|              2.4|01009|
|CN0101100000000

In [5]:
schema = types.StructType([
    types.StructField('LAUS_code',types.StringType(),True),
    types.StructField('state_fips',types.StringType(),True),
    types.StructField('county_fips',types.StringType(),True),
    types.StructField('county_state',types.StringType(),True),
    types.StructField('year',types.IntegerType(),True),
    types.StructField('labor_force',types.IntegerType(),True),
    types.StructField('employed',types.IntegerType(),True),
    types.StructField('unemployed',types.IntegerType(),True),
    types.StructField('unemployment_rate',types.FloatType(),True),
    types.StructField('fips',types.StringType(),True)
])

In [6]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('/Users/alvin/Documents/projects/project_de-zoomcamp/01-ingestion/data/us_employment_2021.csv')

In [7]:
df.schema

StructType(List(StructField(LAUS_code,StringType,true),StructField(state_fips,StringType,true),StructField(county_fips,StringType,true),StructField(county_state,StringType,true),StructField(year,IntegerType,true),StructField(labor_force,IntegerType,true),StructField(employed,IntegerType,true),StructField(unemployed,IntegerType,true),StructField(unemployment_rate,FloatType,true),StructField(fips,StringType,true)))

In [8]:
df.show()

+---------------+----------+-----------+--------------------+----+-----------+--------+----------+-----------------+-----+
|      LAUS_code|state_fips|county_fips|        county_state|year|labor_force|employed|unemployed|unemployment_rate| fips|
+---------------+----------+-----------+--------------------+----+-----------+--------+----------+-----------------+-----+
|CN0100100000000|        01|        001|  Autauga County, AL|2021|      26341|   25599|       742|              2.8|01001|
|CN0100300000000|        01|        003|  Baldwin County, AL|2021|      99427|   96481|      2946|              3.0|01003|
|CN0100500000000|        01|        005|  Barbour County, AL|2021|       8197|    7728|       469|              5.7|01005|
|CN0100700000000|        01|        007|     Bibb County, AL|2021|       8560|    8262|       298|              3.5|01007|
|CN0100900000000|        01|        009|   Blount County, AL|2021|      25127|   24529|       598|              2.4|01009|
|CN0101100000000

In [9]:
df = df.withColumn('state', split(df['county_state'], ', ').getItem(1)) \
    .withColumn('county', split(df['county_state'], ', ').getItem(0))

In [10]:
states = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}

In [11]:
mapping_expr = create_map([lit(x) for x in chain(*states.items())])
df = df.withColumn('state_full', mapping_expr[df.state])

In [12]:
df = df.select("year", "LAUS_code", "fips", "state_fips", "county_fips", "state", "state_full", "county", "labor_force", "employed", "unemployed", "unemployment_rate").show()

+----+---------------+-----+----------+-----------+-----+----------+----------------+-----------+--------+----------+-----------------+
|year|      LAUS_code| fips|state_fips|county_fips|state|state_full|          county|labor_force|employed|unemployed|unemployment_rate|
+----+---------------+-----+----------+-----------+-----+----------+----------------+-----------+--------+----------+-----------------+
|2021|CN0100100000000|01001|        01|        001|   AL|   Alabama|  Autauga County|      26341|   25599|       742|              2.8|
|2021|CN0100300000000|01003|        01|        003|   AL|   Alabama|  Baldwin County|      99427|   96481|      2946|              3.0|
|2021|CN0100500000000|01005|        01|        005|   AL|   Alabama|  Barbour County|       8197|    7728|       469|              5.7|
|2021|CN0100700000000|01007|        01|        007|   AL|   Alabama|     Bibb County|       8560|    8262|       298|              3.5|
|2021|CN0100900000000|01009|        01|        0